# Improving and combining functions

### Outline

* Lambda functions

* Composing and applying functions

* Higher order functions

    * Curried functions
    
    * Partial application

## Lambda functions

The term lamba function comes from the mathematical system called "lambda calculus" developed in the 1930s by Alonzo Church. This system is implemeted in functional programming languages. Lamba functions are often used in Haskell programs. 

### What is a lambda function?

The idea of ​the ​lambda functions or so called anonymous function is that you can define it without naming it.

Here's how a multiplication lambda function $f(x,y)=x*y$ looks like in Haskell:

```haskell
\x y -> x * y
```

The backslash `\` at the beginning denotes the beginning of lambda function after which the arguments follow. And after the arrow `->` comes the main body of the function.

### What is a difference between lambda function and an ordinary function?

* An ordinary function starts with a signature/definition, and it is used somewhere by its name.

* A lambda function doesn't need to have signature and a name, and it can be applied immediately.

Here is an example:

In [ ]:
(\x y -> x * y) 2 3

Lambda functions can also be given *names* and we can call them as ordinary functions (for instance, using `where` construction).

In [ ]:
rectangleArea :: Double -> Double -> Double
rectangleArea width length = mult width length
    where mult = \x y -> x * y

### Type of lambda function

Any function has a type, so does a lambda function. It is a polymorphic function meaning the types are not fixed but are type variables. In the previous example the `mult` has the following type signature show below. This means that the input parameters can be of any type for which the multiplication operation is defined.

```haskell
mult :: a -> a -> a
```

Functions in Haskell are first class citizens which means we can pass them around like any other data structure. For example, you can create a list of functions:

In [ ]:
helloFunction :: String -> String
helloFunction msg = (head functions) msg
    where
        functions = [ \x -> x ++ " world!"
                    , \x -> x ++ " everyone!"
                    ]

helloFunction "Welcome"

The list `functions` has two functions. Two lambda expressions give these two functions, but until the moment they are applied, they do nothing. 

When we apply the head function to this list, we get the first element of the list, that is, the first function. And after that we apply this function to the string `msg`.

## Composing functions

Now we'll explain how to compose functions together and how to get rid of (or just reduce) number of parentheses in the body of the function.

### Example with parentheses

Recall our old function `checkLocalHost` that checks if the argument is a localhost or not and reports the user the result.

In [ ]:
checkLocalhost :: String -> String
checkLocalhost ip =
    if ip == "127.0.0.1"
        then "It’s a localhost!"
        else "No, it's not a localhost."

A function `fourToIP` takes four integers and makes IP address. Then we can use it together with the previous function.

In [ ]:
fourToIP :: Int -> Int -> Int -> Int -> String
fourToIP a b c d = show a ++ "." ++ show b ++ "." ++ show c ++ "." ++ show d

checkLocalhost (fourToIP 127 0 0 1)

Two functions are **composed** here, `checkLocalhost` and `fourToIP`, because `fourToIP` returns `String` and `checkLocalhost` function has `String` at the input. 

Schematically, this can be represented as follows:

$$
    \boxed{\mathrm{(Int, Int, Int, Int)}}
        \xrightarrow{\mathrm{~~~~~~fourToIP~~~~~~}}
    \boxed{\mathrm{String}}
        \xrightarrow{~~~~~~\mathrm{checkLocalhost}~~~~~~}
    \boxed{\mathrm{String}}
$$

The way of composing functions above uses parenthesis. But what will happen if we need to compose three, four or more functions? The number of parenthesis grows and our code becomes less readable. There is another way how to compose functions.

### Function composition and application operator

The composition `checkLocalhost (fourToIP 127 0 0 1)` can be written as

```haskell
checkLocalhost . fourToIP $ 127 0 0 1
```

We have two new standard operators that save us from extra parentheses and making our code easier.

* The function **composition operator** given by a dot character `.`

* The **application operator** `$`

The composition operator combines two functions together (or **composes** them). When we write

```haskell
checkLocalhost . fourToIP
```

instead of two functions `checkLocalhost` and `fourToIP` we obtain a new one `checkLocalhost . fourToIP` called their **composition**. Schematically, this can be represented as follows:

$$
    \boxed{\mathrm{(Int, Int, Int, Int)}}
        \xrightarrow{\mathrm{~~~~~~fourToIP~~~~~~}}
    \boxed{\mathrm{String}}
        \xrightarrow{~~~~~~\mathrm{checkLocalhost}~~~~~~}
    \boxed{\mathrm{String}}
    ~~~~~
    =
    ~~~~~
    \boxed{\mathrm{(Int, Int, Int, Int)}}
        \xrightarrow{\mathrm{~~~~~~checkLocalhost~.~fourToIP~~~~~~}}
    \boxed{\mathrm{String}}
$$


You can use the composition operator to also combine multiple functions together.

The application operator `$` is simply an evaluation of a function at a given value. Without `$` our code will look

```haskell
(checkLocalhost . quadrToIP) 127 0 0 1
```

But we wanted to get rid of the parentheses and for this we need the application operator `$`. Its scheme is simple:

```haskell
FUNCTION $ ARGUMENT
```

where `FUNCTION` is applied to `ARGUMENT`.

#### Remarks about composition and application

* Composition is *right associative*: the function on the right is applied first, and then the one on the left.

* If there are two functions `f :: a -> b` and `g :: b -> c` then their composition as a function can be written without the application operator. For instance, you can write the composition of `checkLocalhost` and `fourToIP` as a new function `isFourLocalhost` using arguments

```haskell
isFourLocalhost :: Int -> Int -> Int -> Int -> String
isFourLocalhost a b c d = checkLocalhost . fourToIP $ a b c d
```

or without them

```haskell
isFourLocalhost :: Int -> Int -> Int -> Int -> String
isFourLocalhost = checkLocalhost . quadrToIP 
```

* The application operator for a composition can be applied before the argument or after the first evaluation. You can write a code

```haskell
checkLocalhost . quadrToIP $ 127 0 0 1
```

in the other alternative form

```haskell
checkLocalhost $ quadrToIP 127 0 0 1
```

## Higher order functions

As we've already learned, functions are data in Haskell. The function that operates on other functions as arguments and / or returns a function as a result is called a **higher order function**. 

Thus, the function composition operator is a higher order function, because

* it takes functions as arguments and

* it returns another function (in the form of lambda function) as a result of its application. 

Using functions as arguments is an extremely common practice in Haskell.

### Curried functions

Under the hood every function in Haskell officially only takes one parameter. So how is it possible that there are functions that take more than one parameter? 

All the functions that accepted several parameters so far have been **curried functions** (the name *curry* derives from the person who popularized the idea: Haskell Curry). 

Let's understand it on an example of the `max` function: it looks like it takes two parameters and returns the one that's bigger. But you can apply the function to only one parameter and you get another function that takes in only one parameter. The following two calls are equivalent:

In [ ]:
max 4 5
(max 4) 5

`max` function is polymorphic and its type is `max :: a -> a -> a`, which can also be written as `max :: a -> (a -> a)`. 

That could be read as: `max` function takes an `a` and returns a function that takes an `a` and returns an `a`. That's why the return type and the parameters of functions are all simply separated with arrows.

The first version `max :: a -> a -> a` is an **uncurried function**, while the second one `max :: a -> (a -> a)` is **curried**.

### Partial application

In the previous example when we provided only one argument to the `max` function we created a new function that takes in only one argument. The application of a function is partial when there are less arguments than expected so we call this process **partial application**.

Using partial application (calling functions with too few parameters) is a neat way to create new functions on the fly so we can pass them to another function or provide them additional data.

Thus, `max 4` creates the following partial function 

```haskell
\x -> max 4 x
```

### Is there a difference between the application operator $ and a function application given by space?

The `$` operator is an infix operator for function application and its signature is

```haskell
($) :: (a -> b) -> a -> b
```

So, given an `a -> b` function and an `a` to apply it to, it gives us a `b`.

On the level of application both `$` and space give the same results:

In [ ]:
sqrt 4
sqrt $ 4

However, if we try the following code, the results will be surprising.

In [ ]:
sqrt 4 + 12
sqrt $ 4 + 12

In the first case the square root is applied first to 4, and only then 12 is added, while in the second one the square root is taken from the sum of 4 and 12. The reason is that the application operator has the lowest priority, and function application - the highest one. Moreover, one of them is right and another one is left associative.

* `$` is an infix operator with right associativity and has the lowest priority possible (Haskell takes everything to its left (that doesn't include another $) and everything to its right (likewise), and effectively puts both sides in parentheses).

* The function application (via space) is left associative and has the highest priority possible (Haskell evaluates each function application first and perform other actions afterwards). 

What that means is you usually see `$` used instead of parentheses to associate things that otherwise wouldn’t.

For any existing or user made operator you can see the information for associativity and priority with the command:
```haskell
:i (<operator_name>)
```

## Summary

In this lesson we've discussed:

* Lambda functions and how to use them.

* Various ways of composing and applying functions.

* What are higher order functions.

* How to curry functions and how does partial application work.